In [2]:
from pathlib import Path
import glob
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import open3d as o3d
import nibabel as nib
from skimage.measure import marching_cubes


In [21]:
DATA_ROOT = Path("/vol/chameleon/projects/mesh_gnn/body_segmentations")
TARGET_ROOT = "/vol/chameleon/projects/mesh_gnn/body_meshes_decimated"
VERTICES = 25000

In [32]:
for file in tqdm(os.listdir(str(DATA_ROOT))):
    _path = str(os.path.join(str(DATA_ROOT), file).replace('\\', '/')) +'/body_mask.nii.gz'
    
    _id = _path[_path[:_path.rfind("/")].rfind("/")+1:_path.rfind("/",0,)]
    print(_path)
    print(_id)
    
    body_segment = nib.load(_path)
    body_segment_data = body_segment.get_fdata()
    verts, faces, _, __ = marching_cubes(body_segment_data, level=0, step_size=1)
    verts = verts/np.array(body_segment_data.shape) 

    mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(verts)),
                                 triangles=o3d.utility.Vector3iVector(np.asarray(faces)))

    decimated_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh, VERTICES-1)

    _target_path=  TARGET_ROOT+"/"+_id+".off"

    o3d.io.write_triangle_mesh( _target_path, decimated_mesh)

    break

  0%|          | 0/44205 [00:00<?, ?it/s]

/vol/chameleon/projects/mesh_gnn/body_segmentations/4105794/body_mask.nii.gz
4105794


  0%|          | 0/44205 [00:07<?, ?it/s]


In [33]:
ex_path = "/vol/chameleon/projects/mesh_gnn/body_meshes_decimated/4105794.off"

In [34]:
decimated = o3d.io.read_triangle_mesh(ex_path)

In [35]:
decimated

TriangleMesh with 13458 points and 24999 triangles.

In [5]:
ex_path= "/vol/chameleon/projects/mesh_gnn/body_segmentations/4105794/body_mask.nii.gz"

In [6]:
body_segment = nib.load(ex_path)
body_segment_data = body_segment.get_fdata()

In [7]:
verts, faces, norms, vals = marching_cubes(body_segment_data, level=0, step_size=1)

In [8]:
verts = verts/np.array(body_segment_data.shape) 

In [9]:
mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(verts)),
                                 triangles=o3d.utility.Vector3iVector(np.asarray(faces)))


In [11]:
decimated_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh, VERTICES)

In [12]:
o3d.io.write_triangle_mesh("test.off", decimated_mesh, print_progress=True )

Writing OFF: [========================================] 100%


True